In [ ]:
!pip install langchain PyJWT

In [ ]:
# 需要安装`PyJWT`
import jwt
import time

from typing import Any, Dict, List, Mapping, Optional

import requests

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens


class ContentHandlerZhipuAPI:
    @classmethod
    def transform_input(
            cls, prompt: str, model_kwargs: Dict[str, Any]
    ) -> Dict[str, Any]:
        return {
            "model": "chatglm_pro",
            "prompt": [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        }

    @classmethod
    def transform_output(cls, response: Any) -> str:
        json = response.json()
        if json['success']:
            return response.json()['data']["choices"][0]['content']
        else:
            return json['msg']

    # 生成智谱的`Token`
    @classmethod
    def generate_token(cls, apikey: str, exp_seconds: int):
        try:
            id, secret = apikey.split(".")
        except Exception as e:
            raise Exception("invalid apikey", e)

        payload = {
            "api_key": id,
            "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
            "timestamp": int(round(time.time() * 1000)),
        }

        return jwt.encode(
            payload,
            secret,
            algorithm="HS256",
            headers={"alg": "HS256", "sign_type": "SIGN"},
        )


class ZhipuAPI(LLM):
    api_url: str

    api_key: str

    model_kwargs: Optional[Dict] = None

    content_handler: ContentHandlerZhipuAPI = ContentHandlerZhipuAPI()

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        _model_kwargs = self.model_kwargs or {}
        return {
            **{"api_url": self.api_url, "api_key": self.api_key},
            **{"model_kwargs": _model_kwargs},
        }

    @property
    def _llm_type(self) -> str:
        return "zhipu_api"

    def _call(
            self,
            prompt: str,
            stop: Optional[List[str]] = None,
            run_manager: Optional[CallbackManagerForLLMRun] = None,
            **kwargs: Any,
    ) -> str:

        _model_kwargs = self.model_kwargs or {}

        payload = self.content_handler.transform_input(prompt, _model_kwargs)

        # 过期时间暂时设为5分钟
        token = ContentHandlerZhipuAPI.generate_token(self.api_key, 5 * 60)

        headers = {
            f'Authorization': f'Bearer {token}'
        }

        try:
            response = requests.post(
                self.api_url,
                headers=headers,
                json=payload,
            )
            text = self.content_handler.transform_output(response)

        except Exception as error:
            raise ValueError(f"Error raised by the service: {error}")

        if stop is not None:
            text = enforce_stop_tokens(text, stop)

        return text


In [ ]:
zhipu_api_key = 'your key'

In [ ]:
llm = ZhipuAPI(
    api_url='https://open.bigmodel.cn/api/paas/v3/model-api/chatglm_lite/invoke',
    api_key=zhipu_api_key,
)

**测试基础LLM**

In [ ]:
result = llm("给我讲一个笑话")

print(f'result = {result}')

result = " 这是一个很简单的笑话:\n\n为什么小鸟站在电线上不会被电到呢？\n\n因为它们不接地啊！"


**测试配合Prompt和Chain使用**

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run({
    'product': "性能卓越的GPU"
}))

" 1.  visiGPU  \n2. gpulabs  \n3. GPU Pro  \n4. lab-GPU  \n5. GPU team  \n6. ProGPUs  \n7. GPU works  \n8. labGPUs  \n9. GPUs limited  \n10. Performance GPUs"


**测试Chain能力**

弱弱的说一句，纽约时报四个字，会触发敏感词校验，不要用

In [ ]:
from langchain.chains import LLMChain, SimpleSequentialChain

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)

synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """你是《豆瓣时间》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《豆瓣时间》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)

review_chain = LLMChain(llm=llm, prompt=prompt_template)

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

review = overall_chain.run("AI简史：从工具到上帝")



> Entering new SimpleSequentialChain chain...
" 标题：《AI 简史：从工具到上帝》\n\n简介：本剧以人工智能 AI 的发展历程为背景，展现了一个从工具到上帝的 AI 进化之路。AI 在科技发展的大背景下应运而生，经历了从模糊、失控到掌控的转变，最终挑战了人类的至高无上的权威。本剧通过对 AI 与人类、科技与道德的碰撞，反思了科技发展给人类带来的启示，展望了 AI 未来发展的趋势与挑战。"
" 《AI 简史：从工具到上帝》是一部引人深思的戏剧作品，以人工智能 AI 的发展历程为背景，展现了一个从工具到上帝的 AI 进化之路。AI 在科技发展的大背景下应运而生，经历了从模糊、失控到掌控的转变，最终挑战了人类的至高无上的权威。本剧通过对 AI 与人类、科技与道德的碰撞，反思了科技发展给人类带来的启示，展望了 AI 未来发展的趋势与挑战。\n\n该剧在戏剧结构上表现出色，通过 AI 与人类之间的交互，展现了科技与道德之间的矛盾与冲突。同时，该剧还深入探讨了 AI 对于人类价值观的影响，以及 AI 技术在未来的发展趋势。在整个戏剧过程中，演员们的表现也非常出色，对于角色的把握十分准确。\n\n在剧情层面，该剧展现了 AI 从工具到上帝的进化过程，刻画了 AI 的发展历程，让人深刻感受到 AI 技术在科技发展中的重要性。此外，该剧还通过对人类对于 AI 技术的恐惧和排斥，展现了科技发展对于人类社会的影响。这些都使得该剧具有很强的思想性和观赏性。\n\n总的来说，《AI 简史：从工具到上帝》是一部非常值得一看的戏剧作品。它通过对 AI 技术与人类、科技与道德的碰撞，展现了科技发展给人类带来的启示，以及 AI 未来发展的趋势与挑战。同时，该剧的演员表现也非常出色，让观众深刻感受到 AI 技术在科技发展中的重要性。"

> Finished chain.
